In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '/home/mm22d016/TransPolymer')
from sklearn.preprocessing import StandardScaler
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaModel
from PolymerSmilesTokenization import PolymerSmilesTokenizer
from dataset import Downstream_Dataset
import torch
import torch.nn as nn
from copy import deepcopy
from torchmetrics import R2Score
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

/home/mm22d016/miniconda3/envs/TransPolymer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = pd.read_csv('../data/Ei.csv')
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=1)

In [3]:
scaler = StandardScaler()
train_data.iloc[:, 1] = scaler.fit_transform(train_data.iloc[:, 1].values.reshape(-1, 1))
test_data.iloc[:, 1] = scaler.transform(test_data.iloc[:, 1].values.reshape(-1, 1))
train_data

,smiles,value
81,*Cc1ccc(-c2ccc(O*)s2)s1,-1.027314
27,*Nc1cccc(C(*)=O)c1,-0.123280
304,*OCCCCONC(*)=O,0.198050
73,*C(=S)c1ccc(C(=S)c2ccc(*)cc2)cc1,-0.657061
219,*C(=O)c1ccc(C(=O)c2ccc(*)cc2)cc1,0.244163
...,...,...
203,*C(=O)c1ccc(-c2ccc(C(*)=S)s2)s1,-0.128381
255,*Nc1ccc(Oc2ccc(*)s2)cc1,-0.889601
72,*Oc1ccc(C(=S)c2ccc(*)cc2)s1,-0.388713
235,*Nc1ccc(C(=O)c2ccc(*)s2)s1,-1.031270


In [4]:
PretrainedModel = RobertaModel.from_pretrained('../ckpt/pretrain.pt')
tokenizer = PolymerSmilesTokenizer.from_pretrained("roberta-base", max_len=411)
train_dataset = Downstream_Dataset(train_data,tokenizer, 411)
test_dataset = Downstream_Dataset(test_data, tokenizer, 411)

Some weights of the model checkpoint at ../ckpt/pretrain.pt were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ../ckpt/pretrain.pt and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer clas

In [5]:
class DownstreamRegression(nn.Module):
    def __init__(self, drop_rate=0.1):
        super(DownstreamRegression, self).__init__()
        self.PretrainedModel = deepcopy(PretrainedModel)
        self.PretrainedModel.resize_token_embeddings(len(tokenizer))
        
        self.Regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(self.PretrainedModel.config.hidden_size, self.PretrainedModel.config.hidden_size),
            nn.SiLU(),
            nn.Linear(self.PretrainedModel.config.hidden_size, 1)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.PretrainedModel(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :] #fingerprint
        output = self.Regressor(logits)
        return output

In [6]:
def test(model, loss_fn, train_dataloader, test_dataloader, device, scaler):

    r2score = R2Score()
    train_loss = 0
    test_loss = 0
    # count = 0
    model.eval()
    with torch.no_grad():
        train_pred, train_true, test_pred, test_true = [],[],[],[]
        for step, batch in enumerate(train_dataloader):
            print('Batch:',step+1)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            prop = batch["prop"].to(device).float()            
            outputs = model(input_ids, attention_mask).float()
            print(outputs.cpu())
            outputs = torch.from_numpy(scaler.inverse_transform(outputs.cpu().reshape(-1, 1)))
            #print("Outputs",outputs)
            prop = torch.from_numpy(scaler.inverse_transform(prop.cpu().reshape(-1, 1))) 
            print("Property",prop)
            loss = loss_fn(outputs.squeeze(), prop.squeeze())
            train_loss += loss.item() * len(prop)
            train_pred.append(outputs.double())
            print('Train_pred list',train_pred)
            train_true.append(prop.double())
            print('Train_true list',train_true)
            print("-------------------------------------------------------------------------------")
            print()
        train_pred = torch.cat(train_pred)
        print('Train Pred tensor',train_pred)
        train_true = torch.cat(train_true)
        print('Train True Tensor',train_true)
    return None

    

In [7]:
model = DownstreamRegression(drop_rate=0.1).to(device)
model = model.double()
loss_fn = nn.MSELoss()

In [8]:
train_dataloader = DataLoader(train_dataset,2,shuffle=False,num_workers=8)
test_dataloader = DataLoader(test_dataset, 2, shuffle=False, num_workers=8)

In [9]:
test(model, loss_fn, train_dataloader, test_dataloader, device, scaler)

Batch: 1
tensor([[0.0326],
        [0.1545]], device='cuda:0')
Property tensor([[5.2494],
        [6.1179]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64)]
Train_true list [tensor([[5.2494],
        [6.1179]], dtype=torch.float64)]
-------------------------------------------------------------------------------

Batch: 2
tensor([[0.1841],
        [0.0241]], device='cuda:0')
Property tensor([[6.4266],
        [5.6051]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64)]
Train_true list [tensor([[5.2494],
        [6.1179]], dtype=torch.float64), tensor([[6.4266],
        [5.6051]], dtype=torch.float64)]
-------------------------------------------------------------------------------

Batch: 3
tensor([[ 0.0022],
        [-0.0038]], device='cuda:0')
Property tensor([[6.4709],
        [5.9175]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.

tensor([[-0.0222],
        [ 0.1591]], device='cuda:0')
Property tensor([[5.5780],
        [7.4502]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float6

tensor([[-0.1131],
        [-0.0439]], device='cuda:0')
Property tensor([[7.0438],
        [6.8199]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float6

tensor([[0.2532],
        [0.2325]], device='cuda:0')
Property tensor([[8.5901],
        [7.7636]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float64)

Train_true list [tensor([[5.2494],
        [6.1179]], dtype=torch.float64), tensor([[6.4266],
        [5.6051]], dtype=torch.float64), tensor([[6.4709],
        [5.9175]], dtype=torch.float64), tensor([[5.6596],
        [4.6632]], dtype=torch.float64), tensor([[5.8964],
        [7.3548]], dtype=torch.float64), tensor([[7.2020],
        [5.9541]], dtype=torch.float64), tensor([[6.1761],
        [7.5966]], dtype=torch.float64), tensor([[6.0770],
        [5.3042]], dtype=torch.float64), tensor([[4.9311],
        [7.5565]], dtype=torch.float64), tensor([[5.6406],
        [5.6689]], dtype=torch.float64), tensor([[5.9845],
        [5.9933]], dtype=torch.float64), tensor([[6.4155],
        [6.4380]], dtype=torch.float64), tensor([[9.0985],
        [9.8385]], dtype=torch.float64), tensor([[5.1674],
        [5.7970]], dtype=torch.float64), tensor([[4.7302],
        [4.7817]], dtype=torch.float64), tensor([[5.5780],
        [7.4502]], dtype=torch.float64), tensor([[6.7464],
        [6.0589]], dt

...

Property tensor([[7.9431],
        [5.9294]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float64), tensor([[6.2150],
        [6.3892]], dtype=torch.flo

Train_true list [tensor([[5.2494],
        [6.1179]], dtype=torch.float64), tensor([[6.4266],
        [5.6051]], dtype=torch.float64), tensor([[6.4709],
        [5.9175]], dtype=torch.float64), tensor([[5.6596],
        [4.6632]], dtype=torch.float64), tensor([[5.8964],
        [7.3548]], dtype=torch.float64), tensor([[7.2020],
        [5.9541]], dtype=torch.float64), tensor([[6.1761],
        [7.5966]], dtype=torch.float64), tensor([[6.0770],
        [5.3042]], dtype=torch.float64), tensor([[4.9311],
        [7.5565]], dtype=torch.float64), tensor([[5.6406],
        [5.6689]], dtype=torch.float64), tensor([[5.9845],
        [5.9933]], dtype=torch.float64), tensor([[6.4155],
        [6.4380]], dtype=torch.float64), tensor([[9.0985],
        [9.8385]], dtype=torch.float64), tensor([[5.1674],
        [5.7970]], dtype=torch.float64), tensor([[4.7302],
        [4.7817]], dtype=torch.float64), tensor([[5.5780],
        [7.4502]], dtype=torch.float64), tensor([[6.7464],
        [6.0589]], dt

Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float64), tensor([[6.2150],
        [6.3892]], dtype=torch.float64), tensor([[6.4303],
        [6.2335]], dt

tensor([[ 0.0621],
        [-0.0940]], device='cuda:0')
Property tensor([[6.4748],
        [7.0413]])
Train_pred list [tensor([[6.2676],
        [6.3847]], dtype=torch.float64), tensor([[6.4132],
        [6.2595]], dtype=torch.float64), tensor([[6.2384],
        [6.2327]], dtype=torch.float64), tensor([[6.2391],
        [6.2100]], dtype=torch.float64), tensor([[6.2227],
        [6.3747]], dtype=torch.float64), tensor([[6.2754],
        [6.2039]], dtype=torch.float64), tensor([[6.1889],
        [6.4368]], dtype=torch.float64), tensor([[6.2845],
        [6.2369]], dtype=torch.float64), tensor([[6.1606],
        [6.1871]], dtype=torch.float64), tensor([[6.3369],
        [6.2030]], dtype=torch.float64), tensor([[6.1838],
        [6.3167]], dtype=torch.float64), tensor([[6.3437],
        [6.1846]], dtype=torch.float64), tensor([[6.2100],
        [6.2991]], dtype=torch.float64), tensor([[6.2371],
        [6.2359]], dtype=torch.float64), tensor([[6.1291],
        [6.3042]], dtype=torch.float6

Train_true list [tensor([[5.2494],
        [6.1179]], dtype=torch.float64), tensor([[6.4266],
        [5.6051]], dtype=torch.float64), tensor([[6.4709],
        [5.9175]], dtype=torch.float64), tensor([[5.6596],
        [4.6632]], dtype=torch.float64), tensor([[5.8964],
        [7.3548]], dtype=torch.float64), tensor([[7.2020],
        [5.9541]], dtype=torch.float64), tensor([[6.1761],
        [7.5966]], dtype=torch.float64), tensor([[6.0770],
        [5.3042]], dtype=torch.float64), tensor([[4.9311],
        [7.5565]], dtype=torch.float64), tensor([[5.6406],
        [5.6689]], dtype=torch.float64), tensor([[5.9845],
        [5.9933]], dtype=torch.float64), tensor([[6.4155],
        [6.4380]], dtype=torch.float64), tensor([[9.0985],
        [9.8385]], dtype=torch.float64), tensor([[5.1674],
        [5.7970]], dtype=torch.float64), tensor([[4.7302],
        [4.7817]], dtype=torch.float64), tensor([[5.5780],
        [7.4502]], dtype=torch.float64), tensor([[6.7464],
        [6.0589]], dt